## Efficient 3D Point Cloud Rendering on the Web

### Why Web Visualization Is Challenging?

#### Data size
- Point clouds can contain billions of points.
- Files easily reach tens or hundreds of gigabytes.
- Full datasets are too large to load into browser memory at once.

#### Rendering Bottlenecks
- Browsers have limited memory and GPU access.
- Real-time rendering struggles with unoptimized data.
- Need to maintain interactive frame rates (30–60 FPS).

#### Optimization Strategies
- Level of Detail (LOD): show fewer points when zoomed out.
- Spatial Indexing: organize data with Octree or KD-tree structures.
- Streaming & Progressive Loading: load only what’s visible.
- WebGL / WebGPU: modern graphics APIs enabling GPU acceleration in browsers.

### Evolution of Graphics APIs — From Desktop to the Web

- **OpenGL** is a cross-platform graphics API used for rendering 2D and 3D content, widely adopted in desktop applications.
- **OpenGL ES (Embedded Systems)** is a simplified version of OpenGL, optimized for mobile devices and embedded systems like smartphones and tablets.
- **WebGL** is a browser-based adaptation of OpenGL ES, allowing 3D graphics to run directly in web browsers without plugins.
- **WebGPU** – A next-generation web graphics API providing modern, low-level GPU access for higher performance, parallel compute, and better control over rendering resources.

### CPU vs GPU Rendering

In both CPU and GPU rendering, the final image is ultimately a 2D array of pixels (a raster) sent to the screen.

The difference is:

- Canvas 2D (CPU): Raster is created in system memory by the CPU and then sent to GPU for display.

- WebGL (GPU): Raster is generated directly in GPU memory, as part of the rendering pipeline — faster, especially for dynamic or complex scenes.

The following examples demonstrate how both the **Canvas 2D API (CPU-based)** and **WebGL (GPU-based)** can fill the canvas with a solid red color. The result is visually the same, but the rendering path is entirely different.

---

#### Minimal CPU Example – Canvas 2D (Software Rendering)

```html
<canvas id="canvas2D" width="300" height="300"></canvas>
<script>
  const canvas = document.getElementById("canvas2D");
  const ctx = canvas.getContext("2d");

  // Set fill color and fill entire canvas
  ctx.fillStyle = "red";
  ctx.fillRect(0, 0, canvas.width, canvas.height);
</script>
```

#### Minimal GPU Example – WebGL (Hardware-Accelerated Rendering)

```html
<canvas id="canvasGL" width="300" height="300"></canvas>
<script>
  const canvas = document.getElementById("canvasGL");
  const gl = canvas.getContext("webgl");

  // Define red color and clear screen
  gl.clearColor(1.0, 0.0, 0.0, 1.0); // RGBA = Red
  gl.clear(gl.COLOR_BUFFER_BIT);
</script>
```
#### CPU based triangle

```html
<canvas id="cpuTriangle" width="300" height="300"></canvas>
<script>
  const canvas = document.getElementById("cpuTriangle");
  const ctx = canvas.getContext("2d");
  ctx.fillStyle = "red";
  ctx.beginPath();
  ctx.moveTo(150, 50);   // top
  ctx.lineTo(75, 250);   // bottom left
  ctx.lineTo(225, 250);  // bottom right
  ctx.closePath();
  ctx.fill();
</script>
```

#### GPU based triangle

```html
<canvas id="glTriangle" width="300" height="300"></canvas>
<script>
  const gl = document.getElementById("glTriangle").getContext("webgl");

  // Vertex shader
  const vs = gl.createShader(gl.VERTEX_SHADER);
  gl.shaderSource(vs, `
    attribute vec2 a_position;
    void main() {
      gl_Position = vec4(a_position, 0, 1);
    }
  `);
  gl.compileShader(vs);

  // Fragment shader
  const fs = gl.createShader(gl.FRAGMENT_SHADER);
  gl.shaderSource(fs, `void main() { gl_FragColor = vec4(1, 0, 0, 1); }`);
  gl.compileShader(fs);

  // Program
  const program = gl.createProgram();
  gl.attachShader(program, vs);
  gl.attachShader(program, fs);
  gl.linkProgram(program);
  gl.useProgram(program);

  // Triangle geometry
  const vertices = new Float32Array([
    0,  0.5,
   -0.5, -0.5,
    0.5, -0.5
  ]);

  const buffer = gl.createBuffer();
  gl.bindBuffer(gl.ARRAY_BUFFER, buffer);
  gl.bufferData(gl.ARRAY_BUFFER, vertices, gl.STATIC_DRAW);

  const loc = gl.getAttribLocation(program, "a_position");
  gl.enableVertexAttribArray(loc);
  gl.vertexAttribPointer(loc, 2, gl.FLOAT, false, 0, 0);

  // Draw
  gl.clearColor(1, 1, 1, 1);
  gl.clear(gl.COLOR_BUFFER_BIT);
  gl.drawArrays(gl.TRIANGLES, 0, 3);
</script>
```
---

### Shaders 

Shaders are small programs that run on the GPU, controlling how graphics are drawn.

Vertex Shader: positions each point or vertex in 3D space.

Fragment Shader: determines the color of each pixel.

Shaders give you full control over rendering — from lighting to color to special effects.

In [6]:
# you can run the HTML/JavaScript examples from Jupyter notebook like this:

from IPython.display import HTML, display

display(HTML(
"""
COPY THE EXAMPLE HERE AND RUN THIS CELL
"""));

## Web 3D Libraries for Point Clouds

In this section, we’ll look at some key libraries that are especially useful for web-based point cloud visualization:

- Babylon.js – a modern 3D engine designed for the next generation of web graphics, already supporting WebGPU for improved performance and flexibility.

- Three.js – a widely used foundational library for web-based 3D graphics, forming the basis for many custom visualization tools, including Potree.

- Potree – an open-source viewer focused on efficient rendering of large LiDAR point clouds, using level-of-detail techniques to handle very dense datasets in the browser.

There are, of course, other libraries such as **CesiumJS**, **deck.gl**, or **iTowns**, which are more oriented toward large-scale geospatial scenes and 3D Tiles rendering. 



### Babylon.js and Three.js

#### Common Principles

Both libraries rely on the same WebGL foundation, so conceptually they do the same thing:

- **GPU rendering:** points are drawn using shaders and vertex buffers.
- **Scene setup:** you need a canvas, a scene, a camera, and a renderer/engine.
- **Point data:** each point has X, Y, Z coordinates and (optionally) color attributes.
- **Camera control:** both offer orbit-style controls for rotating and zooming.
- **Coordinate adaptation:** many point clouds are Z-up, so both need a rotation fix (−90° around X).
- **Bounding sphere:** both compute or derive bounds to frame the camera automatically.

We are going to export our tiledb_pc to json format - this JSON approach works well for smaller point clouds - typically up to a few hundred thousand points. However, JSON becomes inefficient for larger datasets due to its text-based nature and parsing overhead.

For larger point clouds, it is recommended to export the data to a binary format, such as:
- Raw binary arrays (e.g., .bin files for positions and colors), which can be read directly into Float32Array and Uint8Array buffers in JavaScript.
- Or binary PLY/glTF formats, which are standardized and have built-in support in libraries like three.js and Babylon.js.

Binary formats are compact, faster to load, and much better suited for real-time rendering of millions of points in the browser.

When working with very large files (tens or hundreds of millions of points):
- Avoid loading the entire dataset at once. Instead, stream data in chunks or use range-based requests to load subsets of the binary file as needed.
- Organize your dataset using an octree or spatial indexing scheme. This allows you to:
   - Load only the visible parts of the point cloud (view-dependent level of detail).
   - Quickly query points within specific regions of interest.
   - Improve rendering performance by reducing GPU memory use.

In practice, you can preprocess your data into an octree structure (for example, with [PotreeConverter](https://github.com/potree/PotreeConverter), [entwine](https://entwine.io/en/latest/), or a custom Python script) and then load progressively based on camera position. Both three.js and Babylon.js can be adapted to dynamically request and render octree tiles for efficient visualization of large point clouds.

In [1]:
#dump the data to JSON file
import tiledb
import pandas as pd
import os

os.chdir("/media/lubuntu/USBDATA")

array_name = "./results/tiledb_pc"
A = tiledb.open(array_name)

# Valid domain-sliced query
df = A.query(attrs=('Red', 'Green','Blue')).df[
    461150:461250,  # X
    100300:100400,  # Y
    231:300         # Z
]

data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'],
    'Green': df['Green'],
    'Blue': df['Blue']
}

# df already has X,Y,Z, Red,Green,Blue
df[['X','Y','Z','Red','Green','Blue']].to_json("./web/examples/data/points.json", orient="records")

"""#binary export
pos = df[['X','Y','Z']].to_numpy(dtype=np.float32, copy=False)
col = df[['Red','Green','Blue']].to_numpy(dtype=np.uint8, copy=False)

pos.tofile("./web/examples/data/points.positions.bin")  # float32, length = N*3
col.tofile("./web/examples/data/points.colors.bin")     # uint8,   length = N*3
"""

'#binary export\npos = df[[\'X\',\'Y\',\'Z\']].to_numpy(dtype=np.float32, copy=False)\ncol = df[[\'Red\',\'Green\',\'Blue\']].to_numpy(dtype=np.uint8, copy=False)\n\npos.tofile("./web/examples/data/points.positions.bin")  # float32, length = N*3\ncol.tofile("./web/examples/data/points.colors.bin")     # uint8,   length = N*3\n'

From the `web` subfolder start the HTTP server (we are going to start it from the terminal to see the output and to shut it down more easily)

cd /media/lubuntu/USBDATA/web;
python -m http.server 8000

In the browser go to the http://127.0.0.1:8000/ and then to the [examples](http://127.0.0.1:8000/examples) folder where you can find [basic example for babylon.js](http://127.0.0.1:8000/examples/babylon.js.html) and [basic example for three.js](http://127.0.0.1:8000/examples/three.js.html)

## Potree

- https://github.com/potree/potree
- https://github.com/potree/PotreeConverter

Python wrapper: https://github.com/centreborelli/pypotree

Let's draw a cube of 3D points (example from the github):

In [2]:

import pypotree 
import numpy as np
import os

os.chdir("/media/lubuntu/USBDATA/web")
xyz = np.random.random((100000,3))
cloudpath = pypotree.generate_cloud_for_display(xyz)
print(cloudpath)
pypotree.display_cloud(cloudpath) # this does not work from file system - it is to be served with http server

from IPython.display import IFrame
IFrame(f"http://127.0.0.1:8000/point_clouds/{cloudpath}.html", width=1000, height=1000)

/home/lubuntu/micromamba/envs/geo_env/lib/python3.11/site-packages/bin/PotreeConverter .tmp.txt -f xyz -o point_clouds -p 327995 --material ELEVATION --edl-enabled --overwrite
== params ==
source[0]:         	.tmp.txt
outdir:            	point_clouds
spacing:           	0
diagonal-fraction: 	200
levels:            	-1
format:            	xyz
scale:             	0
pageName:          	327995
projection:        	

AABB: 
min: [7,65679e-06, 5,6661e-06, 1,63266e-05]
max: [0,999998, 0,99999, 0,999998]
size: [0,99999, 0,999985, 0,999981]

cubic AABB: 
min: [7,65679e-06, 5,6661e-06, 1,63266e-05]
max: [0,999998, 0,999996, 1,00001]
size: [0,99999, 0,99999, 0,99999]

spacing calculated from diagonal: 0,00866017
READING:  .tmp.txt
closing writer

conversion finished
100000 points were processed and 100000 points ( 100% ) were written to the output. 
duration: 0,884s
327995


### PotreeConverter

In [14]:
# convert single file and generate web page viewer
!/media/lubuntu/USBDATA/web/potree/PotreeConverter_linux_x64/PotreeConverter /media/lubuntu/USBDATA/copc/GKOT_461_100.copc.laz -o /media/lubuntu/USBDATA/web/potree_converted -p GKOT_461_100

== params ==
source[0]:         	/media/lubuntu/USBDATA/copc/GKOT_461_100.copc.laz
outdir:            	/media/lubuntu/USBDATA/web/potree_converted
spacing:           	0
diagonal-fraction: 	200
levels:            	-1
format:            	
scale:             	0
pageName:          	GKOT_461_100
projection:        	

processing following attributes: 
POSITION_CARTESIAN
RGBA
classification
gps-time
intensity
number of returns
return number
source id

AABB: {
	"min": [460999.824500, 100000.000000, 231.304750],
	"max": [461999.999750, 101000.045500, 382.628000],
	"size": [1000.175250, 1000.045500, 151.323250]
}

cubicAABB: {
	"min": [460999.824500, 100000.000000, 231.304750],
	"max": [461999.999750, 101000.175250, 1231.480000],
	"size": [1000.175250, 1000.175250, 1000.175250]
}

total number of points: 16236043
spacing calculated from diagonal: 8,66177
READING:  /media/lubuntu/USBDATA/copc/GKOT_461_100.copc.laz
INDEXING: 1000000 of 16236043 processed (6%); 1000000 written; 7,739 seconds passed

In [17]:
#convert single file and generate web page, add projection
!/media/lubuntu/USBDATA/web/potree/PotreeConverter_linux_x64/PotreeConverter /media/lubuntu/USBDATA/copc/GKOT_461_100.copc.laz -o /media/lubuntu/USBDATA/web/potree_converted -p GKOT_461_100_with_projection --projection "+proj=tmerc +lat_0=0 +lon_0=15 +k=0.9999 +x_0=500000 +y_0=-5000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs +type=crs"

== params ==
source[0]:         	/media/lubuntu/USBDATA/copc/GKOT_461_100.copc.laz
outdir:            	/media/lubuntu/USBDATA/web/potree_converted
spacing:           	0
diagonal-fraction: 	200
levels:            	-1
format:            	
scale:             	0
pageName:          	GKOT_461_100_with_projection
projection:        	+proj=tmerc +lat_0=0 +lon_0=15 +k=0.9999 +x_0=500000 +y_0=-5000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs +type=crs

processing following attributes: 
POSITION_CARTESIAN
RGBA
classification
gps-time
intensity
number of returns
return number
source id

AABB: {
	"min": [460999.824500, 100000.000000, 231.304750],
	"max": [461999.999750, 101000.045500, 382.628000],
	"size": [1000.175250, 1000.045500, 151.323250]
}

cubicAABB: {
	"min": [460999.824500, 100000.000000, 231.304750],
	"max": [461999.999750, 101000.175250, 1231.480000],
	"size": [1000.175250, 1000.175250, 1000.175250]
}

total number of points: 16236043
spacing calculated from diagonal: 8,661

In [18]:
#convert all files inside the data directory, add projection
!/media/lubuntu/USBDATA/web/potree/PotreeConverter_linux_x64/PotreeConverter /media/lubuntu/USBDATA/copc -o  /media/lubuntu/USBDATA/web/potree_converted -p all_files --projection "+proj=tmerc +lat_0=0 +lon_0=15 +k=0.9999 +x_0=500000 +y_0=-5000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs +type=crs"

== params ==
source[0]:         	/media/lubuntu/USBDATA/copc
outdir:            	/media/lubuntu/USBDATA/web/potree_converted
spacing:           	0
diagonal-fraction: 	200
levels:            	-1
format:            	
scale:             	0
pageName:          	all_files
projection:        	+proj=tmerc +lat_0=0 +lon_0=15 +k=0.9999 +x_0=500000 +y_0=-5000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs +type=crs

processing following attributes: 
POSITION_CARTESIAN
RGBA
classification
gps-time
intensity
number of returns
return number
source id

AABB: {
	"min": [460999.791250, 100000.000000, 231.304750],
	"max": [463000.045500, 102000.125000, 421.387000],
	"size": [2000.254250, 2000.125000, 190.082250]
}

cubicAABB: {
	"min": [460999.791250, 100000.000000, 231.304750],
	"max": [463000.045500, 102000.254250, 2231.559000],
	"size": [2000.254250, 2000.254250, 2000.254250]
}

total number of points: 63816694
spacing calculated from diagonal: 17,3227
READING:  /media/lubuntu/USBDATA/copc/

---
This is what PotreeConverter GitHub page is saying about its features:

https://github.com/potree/PotreeConverter

> Version 2.0 is a complete rewrite with following differences over the previous version 1.7:
> - About 10 to 50 times faster than PotreeConverter 1.7 on SSDs.
> - Produces a total of 3 files instead of thousands to tens of millions of files. The reduction of the number of files improves file system operations such as copy, delete and upload to servers from hours and days to seconds and minutes.
> - Better support for standard LAS attributes and arbitrary extra attributes. Full support (e.g. int64 and uint64) in development.
> - Optional compression is not yet available in the new converter but on the roadmap for a future update.


In [19]:
#convert single file and generate web page, add projection
!/media/lubuntu/USBDATA/web/potree/PotreeConverter_2.1.1_x64_linux/PotreeConverter /media/lubuntu/USBDATA/copc/GKOT_461_100.copc.laz -o /media/lubuntu/USBDATA/web/potree_converted -p GKOT_461_100_projected --projection "+proj=tmerc +lat_0=0 +lon_0=15 +k=0.9999 +x_0=500000 +y_0=-5000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs +type=crs"

#threads: 6
#paths: 1
ERROR: currently unsupported LAS format: 8


In [20]:
!pdal translate /media/lubuntu/USBDATA/copc/GKOT_461_100.copc.laz /media/lubuntu/USBDATA/web/GKOT_461_100_id_7.laz --writers.las.dataformat_id=7

In [22]:
!/media/lubuntu/USBDATA/web/potree/PotreeConverter_2.1.1_x64_linux/PotreeConverter /media/lubuntu/USBDATA/web/GKOT_461_100_id_7.laz -o /media/lubuntu/USBDATA/web/potree_converted -p GKOT_461_100_potree_converter_2p1 --projection "+proj=tmerc +lat_0=0 +lon_0=15 +k=0.9999 +x_0=500000 +y_0=-5000000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs +type=crs"

#threads: 6
#paths: 1

output attributes: 
name                              offset    size
position                               0      12
intensity                             12       2
return number                         14       1
number of returns                     15       1
classification flags                  16       1
classification                        17       1
user data                             18       1
scan angle                            19       2
point source id                       21       2
gps-time                              23       8
rgb                                   31       6
                                              37
cubicAABB: {
	"min": [460999.820000, 100000.000000, 231.300000],
	"max": [462000.000000, 101000.180000, 1231.480000],
	"size": [1000.180000, 1000.180000, 1000.180000]
}
#points: 16'236'043
total file size: 107.0 MB
target directory: '/media/lubuntu/USBDATA/web/potree_converted/pointclouds/GKOT_461_100_potree_converter_

There is some strange output if we look at the generated file with our simple python HTTP server. This is because the PotreeConverter 2.x provides single file to be consumed via HTTP range requests and the basic python HTTP server we are using does not know how to handle this type of requests.

Run another HTTP server which has support for HTTP range requests:

http-server /media/lubuntu/USBDATA/web -p 8080

Similar mechanism (with HTTP range requests) is also provided in PotreeViewer for copc files - we modified the default Potree_1.8.2 example to show `GKOT_461_100.copc.laz`. The modified example can be found here: http://127.0.0.1:8080/potree/Potree_1.8.2/examples/copc.html